## 머신러닝기반 데이터분석_정아영

https://www.kaggle.com/elenapetrova/time-series-analysis-and-forecasts-with-prophet 

위 커널의 따라하기 및 한국어 번역 자료입니다. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

목표:
*     데이터 탐색 (ECDF, 결측값 처리 등)
*     스토어 유형별 분석 및 스토어 활동의 상관 분석
*     광범위한 시계열 분석(계절 분해,추세,자기상관) 수행
*     Prophet을 사용하여 다음 6주간의 판매 예측(페이스북 방법론)
> ※Prophet은 페이스북에서 개발한 시계열 예측 패키지이다. 

이 노트북은 시계열분석에 초점을 맞춘다. 중요한 주제는 아직 다뤄지지 않았다. 나는 최근 페이스북에서 소개한 새로운 방법론 Prophet을 사용하여 다음 6주의 판매를 예측한다.
이 방법론은 휴일을 위한 모델링의 멋진 특성을 가지고 있다. 마지막으로 나는 또한 계절ARIMA와 Prophet을 사용한 예측의 장단점에 대해 논의한다.

보통 그렇듯이 우리는 데이터의 현재 동향과 패턴을 보여주는 주요 지표의 탐색적 데이터분석부터 시작하여, 추가 (인과)분석을 위한 견고한 기초를 마련한다.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# loading packages 패키지 불러오기
# basic + dates 기본+날짜
import numpy as np
import pandas as pd
from pandas import datetime

# data visualization 데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs 고급시각화
%matplotlib inline

# statistics 통계
from statsmodels.distributions.empirical_distribution import ECDF

# time series analysis 시계열 분석
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# prophet by Facebook
from fbprophet import Prophet

In [ ]:
# importing train data to learn 학습 데이터
train = pd.read_csv("../input/rossmann-store-sales/train.csv",parse_dates = True, low_memory = False, index_col = 'Date')

# additional store data 추가 스토어 데이터
store = pd.read_csv("../input/rossmann-store-sales/store.csv", 
                    low_memory = False)
# time series as indexes
train.index

# 탐색적 데이터 분석

첫번째 섹션에서 train과 store데이터를 통해, 결측치를 처리하고 추가 분석을 위한 새로운 특징을 생성한다.

In [ ]:
# first glance at the train set: head and tail 학습 데이터 훑어보기
print("In total: ", train.shape)
train.head(5)

간단한 설명:
* Sales: 주어진 날짜에 대한 목표액 (목표변수)
* Customers: 주어진 날짜의 고객 수
* Open: 스토어의 오픈여부: 0=closed,1=open
* Promo: 해당 날짜의 프로모션 실행 여부
* StateHoliday: 공휴일. 보통 예외없이 거의 모든 스토어가 공휴일에는 문을 닫는다.
* SchoolHoliday: 공립학교가 닫은 날에 영향을 받았는지에 대한 여부를 나타낸다.

우리는 시계열 데이터를 다루고 있기 때문에 추가 분석을 위해 날짜를 추출하는 것이 도움이 될 것이다. 또한 데이터셋에서 상관관계가 있는 두 가지는 새로운 특징으로 결합될 수 있다.

In [ ]:
# data extraction 데이터 추출
train['Year'] = train.index.year
train['Month'] = train.index.month
train['Day'] = train.index.day
train['WeekOfYear'] = train.index.weekofyear

# adding new variable 새로운 변수 추가
train['SalePerCustomer'] = train['Sales']/train['Customers']
train['SalePerCustomer'].describe()

비록 판매량이 0인 날이 있기는 하지만, 평균적으로 고객들은 하루에 약 9.50달러를 소비한다.

**ECDF: empirical cumulative distribution function (경험적 누적 분포 함수)
**

데이터의 지속적 변수에 대한 첫 느낌을 얻기 위해 ECDF를 도표화 하였다.

> ECDF는 데이터 분포의 적합(fit)을 평가하거나 서로 다른 여러 표본 분포를 비교할 때 사용

In [ ]:
sns.set(style = "ticks")# to format into seaborn 
c = '#386B7F' # basic color for plots
plt.figure(figsize = (12, 6))

plt.subplot(311)  #3x1중 1번째 위치
cdf = ECDF(train['Sales'])
plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
plt.xlabel('Sales'); plt.ylabel('ECDF');

# plot second ECDF  
plt.subplot(312)
cdf = ECDF(train['Customers'])
plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
plt.xlabel('Customers');

# plot second ECDF  
plt.subplot(313)
cdf = ECDF(train['SalePerCustomer'])
plt.plot(cdf.x, cdf.y, label = "statmodels", color = c);
plt.xlabel('Sale per Customer');

plt.tight_layout()    #그래프 간 간격 조정

데이터의 약20%는 우리가 처리해야 할 매출/고객의 제로값을 가지고 있었고, 거의 80%의 시간일별 매출은 1000미만이었다. 그렇다면 제로매출은 어떨까, 오직 스토어가 문을 닫은 것 때문일까?

# 결측값
폐점한 스토어와 제로매출 스토어

In [ ]:
# closed stores 폐점 스토어
train[(train.Open == 0) & (train.Sales == 0)].count()

데이터 안에 172817개의 폐점 매장이 있다. 이것은 관측 총량의 약 10%에 해당한다. 편향된 예측을 피하기 위해 이 값들을 drop할 것이다.

매출이 전혀 없는 오픈 매장은 어떨까?

In [ ]:
# opened stores with zero sales 매출이 없는 스토어
zero_sales = train[(train.Open != 0) & (train.Sales == 0)]
print("In total: ", zero_sales.shape)
zero_sales.head(5)

흥미롭게도, 영업일에 매출이 없는 오픈 매장이 있다. 데이터에는 54일밖에 없기 때문에, 우리는 예를 들어 발현과 같은 외부 요인이 관련되어 있었다고 추정할 수 있다.

In [ ]:
print("폐점매장과 매출이 없는 매장을 제외하여 다시 학습데이터(Train)를 만들어준다.")
train = train[(train["Open"] != 0) & (train['Sales'] != 0)]

print("In total: ", train.shape)

In [ ]:
# additional information about the stores 스토어 정보
store.head()

* Store: 각 스토어에 대한 고유 ID
* StoreType: 4가지 스토어모델 구분(a,b,c,d)
* Assortment: 분류레벨 (a=basic,b=extra,c=extended)
* CompetitionDistance: 가장 가까운 경쟁업체까지의 거리(미터)
* CompetitionOpenSince[Month/Year]: 가장 가까운 경쟁업체가 문을 연 대략적인 연,월
* Promo2: 일부 매장을 위한 지속적인 프로모션 (0=참가하지 않음,1=참가중임)
* Promo2Since[Year/Week]: 프로모션에 참가하기 시작한 연도와 캘린더주
* PromoInterval: 프로모션이 시작되는 연속 간격, 프로모션이 시작된 월 이름

In [ ]:
# missing values?
store.isnull().sum()

처리해야 할 결측치가 있는 변수는 많지 않다.

In [ ]:
# missing values in CompetitionDistance
store[pd.isnull(store.CompetitionDistance)]

분명히 이 정보는 단순히 데이터에서 누락되었다. 특정한 패턴이 관찰되지 않는다. 이 경우에, NaN을 중간값(평균보다 두배 적은)으로 대체하는 것이 타당하게 느껴진다.

In [ ]:
# fill NaN with a median value (skewed distribuion) 결측치를 중간값으로 대체
store['CompetitionDistance'].fillna(store['CompetitionDistance'].median(), inplace = True)

In [ ]:
# no promo = no information about the promo?
_ = store[pd.isnull(store.Promo2SinceWeek)]
_[_.Promo2 != 0].shape
#Promo2SinceWeek의 결측치 중에서 Promo2가 0이 아닌 것 조회

Promo2SinceWeek은 Promo2가 없으면 정보가 없는 것이다. 이 결측치를 0으로 대체해준다. CompetitionOpenSinceMonth와 CompetitionOpenSinceYear도 마찬가지이다.

In [ ]:
# replace NA's by 0
store.fillna(0, inplace = True)

In [ ]:
print("train데이터와 store데이터를 조인")

# inner join을 해줌으로서 관찰 값들만 합쳐줌. Store(스토어 고유ID)를 기준으로 병합
train_store = pd.merge(train, store, how = 'inner', on = 'Store')

print("In total: ", train_store.shape)
train_store.head()

# 상점 유형
이번 섹션은 StoreType의 다양한 레벨과 주요 측정지표인 Sales가 이들 사이에 어떻게 분배되어 있는지 살펴본다. 

In [ ]:
train_store.groupby('StoreType')['Sales'].describe()

StoreType B가 판매평균은 가장 높지만, 데이터는 가장 적다. 매출과 고객의 전체 합계를 출력하여 어떤 상점유형이 가장 많이 팔리고 붐비는지 확인해보자.

In [ ]:
train_store.groupby('StoreType')['Customers', 'Sales'].sum()

Type A가 분명하다. Type D는 매출과 고객에서 모두 2위를 하고 있다.

날짜와 기간에 대해서는 어떨까? Seaborn의 facet grid는 이런 작업에 가장 적합한 도구이다.

In [ ]:
# sales trends 월별 매출액
sns.factorplot(data = train_store, x = 'Month', y = "Sales", 
               col = 'StoreType', # per store type in cols
               palette = 'plasma',
               hue = 'StoreType',
               row = 'Promo', # per promo in the store in rows
               ) 

In [ ]:
# sales trends 월별 고객
sns.factorplot(data = train_store, x = 'Month', y = "Customers", 
               col = 'StoreType', # per store type in cols
               palette = 'plasma',
               hue = 'StoreType',
               row = 'Promo', # per promo in the store in rows
               color = c) 

모든 매장 유형은 같은 동향을 따르지만 (B의 경우)(첫)프로모션 참여와 스토어타입 자체에 따라 규모가 다르다.

이미 이 시점에서, 우리는 판매량이 크리스마스 연휴에 확대되는 것을 볼 수 있다. 하지만 계절성과 동향에 대해서는 나중에 시계열 분석 섹션에서 설명하겠다.

In [ ]:
# sale per customer trends
sns.factorplot(data = train_store, x = 'Month', y = "SalePerCustomer", 
               col = 'StoreType', # per store type in cols
               palette = 'plasma',
               hue = 'StoreType',
               row = 'Promo', # per promo in the store in rows
               color = c) 

위의 차트에서 Type B가 가장 매출이 높고 성능이 좋은 것으로 나왔지만, 실제로는 사실이 아니다. 가장 높은 SalePerCustomer값은 프로모션할 경우 12유로, 없을 경우 10유로인 Type D로 보여진다. Type A와 C는 약 9유로이다. 

Type B의 낮은 SalePerCustomer 값은 고객차트를 설명한다: 본질적으로 작은 물건(또는 적은 양)을 위해 쇼핑하는 사람들이 많다. 더하여 이 StoreType은 전체적으로 기간동안 가장 적은 매출과 고객을 발생시키는 것으로 보인다.

In [ ]:
# customers
sns.factorplot(data = train_store, x = 'Month', y = "Sales", 
               col = 'DayOfWeek', # per store type in cols
               palette = 'plasma',
               hue = 'StoreType',
               row = 'StoreType', # per store type in rows
               color = c) 

Type C의 상점이 일요일에 모두 문을 닫는 반면, 다른 상점들은 대부분 문을 연다는 것을 볼 수 있다. 흥미롭게도, Type D는 10월부터 12월까지만 일요일에 문을 닫는다.

그런데 일요일에 여는 상점들은 어떤 곳일까?

In [ ]:
# stores which are opened on Sundays 일요일에 여는 상점
train_store[(train_store.Open == 1) & (train_store.DayOfWeek == 7)]['Store'].unique()   #unique는 중복없이 유일값 조회

예비 데이터 분석을 완료하기 위해 경쟁업체 및 프로모션이 열린 기간을 설명하는 변수를 추가할 수 있다.

In [ ]:
# competition open time (in months)
train_store['CompetitionOpen'] = 12 * (train_store.Year - train_store.CompetitionOpenSinceYear) + \
        (train_store.Month - train_store.CompetitionOpenSinceMonth)
    
# Promo open time
train_store['PromoOpen'] = 12 * (train_store.Year - train_store.Promo2SinceYear) + \
        (train_store.WeekOfYear - train_store.Promo2SinceWeek) / 4.0

# average PromoOpen time and CompetitionOpen time per store type
train_store.loc[:, ['StoreType', 'Sales', 'Customers', 'PromoOpen', 'CompetitionOpen']].groupby('StoreType').mean()

가장 많이 팔리고 붐비는 Type A는 경쟁업체에 가장 노출되지 않는 것 중 하나로 보인다. 대신에 Type B가 가장 프로모션 기간이 길고 경쟁업체 노출이 많다. 

# 상관분석

데이터에 새로운 변수를 추가하는 것을 마쳤으므로, 전체적인 상관관계를 seaborn heatmap을 통해 도표화하여 확인해본다.

In [ ]:
# Compute the correlation matrix 상관관계 비교
# exclude 'Open' variable 변수 제외
corr_all = train_store.drop('Open', axis = 1).corr()

# Generate a mask for the upper triangle 삼각형 마스크를 생성
mask = np.zeros_like(corr_all, dtype = np.bool)  # np.zeros_like는 기존 array와 동일한 형태에서 0을 반환 
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize = (11, 9))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_all, mask = mask,
            square = True, linewidths = .5, ax = ax, cmap = "BuPu")      
plt.show()

앞에서 언급했듯이, 우리는 상점의 판매량과 고객 사이에 강한 긍정적 상관관계를 가지고 있다. 또한, 프로모션(Promo=1)진행과 고객 수와의 긍정적인 상관관계도 엿볼 수 있다.

그러나 연이은 프로모션(Promo2=1)을 계속하는 순간 고객 및 매출 건수는 그대로 유지되거나 감소하는 것으로 보이며, 이는 히트맵에 옅은 마이너스 상관관계로 설명된다. 프로모션 유무와 요일 간에도 동일한 부정적 상관관계가 관찰된다.

In [ ]:
# sale per customer trends
sns.factorplot(data = train_store, x = 'DayOfWeek', y = "Sales", 
               col = 'Promo', 
               row = 'Promo2',
               hue = 'Promo2',
               palette = 'RdPu') 

* 프로모션이 없을 경우(Promo=0,Promo2=0), 판매는 일요일에 정점을 찍는 경향이 있다. 비록 우리는 Type C가 일요일에는 작동하지 않는다는 것을 알아두어야 한다. 그래서 주로 StoreType A, B, D의 데이터다.
* 반대로 프로모션을 진행하는 상점들은 월요일 매출을 차지하는 경향이 있다. 이 사실은 로스만 마케팅 캠페인에 좋은 지표가 될 수 있다. 같은 트렌드는 두 가지 프로모션을 동시에 진행하는 스토어(Promo=1,Promo2=1)를 따른다.
* Promo2만으로는 판매 금액의 큰 변화와 상관관계가 없어 보인다. 이것은 또한 위의 heatmap에서 옅은 파란색부분에 의해 증명될 수 있다.

# EDA의 결론

* 가장 많이 팔리고 붐피는 상점유형은 A이다.
* 'Sale per Customer'가 가장 높은 상점유형 D는 상위 고객 차트를 가리킨다. 이 사실로부터 이익을 얻기 위해 로스만은 다양제품을 생산하는 것을 고려해볼 수 있다. 
* Type B의 낮은 Sale per Customer 금액은 사람들이 본질적으로 '작은' 것들을 위해 그 곳에서 쇼핑을 한다는 것을 나타낸다. 이 상점유형은 전체 기간 동안 가장 적은 매출과 고객을 창출했음에도 불구하고 큰 잠재력을 보여주고 있다.
* 고객은 프로모션(Promo)이 하나 있는 월요일과 프로모션이 전혀 없는 일요일(Promo=0,Promo2=0)에 더 많이 구매하는 경향이 있다.
* Promo2만으로는 매출금액의 변화에 상관관계가 없어 보인다.

# 상점 유형별 시계열분석

시계열은 일반 회귀 문제와 무엇이 다른가?
* 그것은 시간 의존적이다. 관측치가 독립적이라는 선형 회귀의 기본 가정은 이 경우에 적합하지 않다.
* 증가하거나 감소하는 경향과 함께, 대부분의 시계열은 계절성 경향의 어떤 형태, 즉 특정 시간 프레임에 특정한 변화를 가진다. 예를 들어 이 데이터셋에서 크리스마스연휴를 들 수 있다.

우리는 개별 매장 대신 매장 유형에 대한 시계열 분석을 구축한다. 이 접근방식의 주요 장점은 데이터셋에서 다양한 동향과 계절적 특성을 설명하는 표시의 단순성 및 전체적인 설명이다.

이 섹션에서는 시계열 데이터(동향, 계절적 특성 및 자기 상관)를 분석한다. 보통 분석의 마지막에 계절적 ARIMA(자동 회귀 통합 이동 평균) 모델을 개발할 수 있지만, 지금은 그것이 주된 초점이 되지 않을 것이다. 대신 우리는 자료를 이해하려고 노력하며, 나중에 Prophet방법론을 사용하여 예측을 내놓는다.

**계절 특성**

각 그룹을 대표하기 위해 상점 유형으로부터 4가지를 가져온다.
* Store number 2 for StoreType A
* Store number 85 for StoreType B,
* Store number 1 for StoreType C
* Store number 13 for StoreType D.

또한 현재의 추세를 보다 명확하게 보기 위해 재샘플링 방법을 사용하여 데이터를 며칠에서 몇 주까지 다운샘플링하는 것이 타당하다.

> resample 연산을 쓰면 시간 간격을 재조정하는 리샘플링(resampling)이 가능하다. 이 때 시간 구간이 작아지면 데이터 양이 증가한다고 해서 업-샘플링(up-sampling)이라 하고 시간 구간이 커지면 데이터 양이 감소한다고 해서 다운-샘플링(down-sampling)이라 부른다.

In [ ]:
# preparation: input should be float type
train['Sales'] = train['Sales'] * 1.0

# store types
sales_a = train[train.Store == 2]['Sales']
sales_b = train[train.Store == 85]['Sales'].sort_index(ascending = True)
# solve the reverse order
sales_c = train[train.Store == 1]['Sales']
sales_d = train[train.Store == 13]['Sales']

f, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (12, 13))

# store types
sales_a.resample('W').sum().plot(color = c, ax = ax1)
sales_b.resample('W').sum().plot(color = c, ax = ax2)
sales_c.resample('W').sum().plot(color = c, ax = ax3)
sales_d.resample('W').sum().plot(color = c, ax = ax4)

Type A와 C의 소매판매는 크리스마스 시즌에 절정을 이뤘다가 시즌이 지나면 감소하는 경향이 있다. Type D도 같은 추세를 보였을 수 있지만 2014년 7월부터 2015년 1월까지 문을 닫아 관련 정보가 없다.

**연도별 추세**

In [ ]:
f, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (12, 13))

# monthly
decomposition_a = seasonal_decompose(sales_a, model = 'additive', freq = 365)
decomposition_a.trend.plot(color = c, ax = ax1)

decomposition_b = seasonal_decompose(sales_b, model = 'additive', freq = 365)
decomposition_b.trend.plot(color = c, ax = ax2)

decomposition_c = seasonal_decompose(sales_c, model = 'additive', freq = 365)
decomposition_c.trend.plot(color = c, ax = ax3)

decomposition_d = seasonal_decompose(sales_d, model = 'additive', freq = 365)
decomposition_d.trend.plot(color = c, ax = ax4)

**자기 상관**

시계열 분석의 다음 단계는 자기 상관 함수(ACF)와 부분 자기 상관 함수(PACF) 차트를 검토하는 것이다.

ACF는 그 자체의 지연된 버전과의 시계열 사이의 상관관계를 측정한 것이다. 예를 들어, 지연 시간 5에서 ACF는 't1'…'tn'시리즈를  't1-5'…'tn-5'(t1-5 및 tn이 end points)의 시리즈와 비교한다.

반면에, PACF는 중간 비교에 의해 설명되는 변동을 제거한 후, 그 자체의 지연된 버전과 시계열 간의 상관 관계를 측정한다. 예를 들어, 5차 시기에는 상관관계를 확인하지만 1~4차 시차로 이미 설명되어 있는 효과는 제거한다.

In [ ]:
# figure for subplots
plt.figure(figsize = (12, 8))

# acf and pacf for A
plt.subplot(421); plot_acf(sales_a, lags = 50, ax = plt.gca(), color = c)
plt.subplot(422); plot_pacf(sales_a, lags = 50, ax = plt.gca(), color = c)

# acf and pacf for B
plt.subplot(423); plot_acf(sales_b, lags = 50, ax = plt.gca(), color = c)
plt.subplot(424); plot_pacf(sales_b, lags = 50, ax = plt.gca(), color = c)

# acf and pacf for C
plt.subplot(425); plot_acf(sales_c, lags = 50, ax = plt.gca(), color = c)
plt.subplot(426); plot_pacf(sales_c, lags = 50, ax = plt.gca(), color = c)

# acf and pacf for D
plt.subplot(427); plot_acf(sales_d, lags = 50, ax = plt.gca(), color = c)
plt.subplot(428); plot_pacf(sales_d, lags = 50, ax = plt.gca(), color = c)

plt.tight_layout()
plt.show()

이 차트를 수평으로 읽을 수 있다. 각 수평 쌍은 A에서 D까지 하나의 'StoreType'을 위한 것이다. 일반적으로 이러한 그림은 시리즈 자체와의 상관관계를 보여주고 있으며, 시리즈 자체와의 상관관계는 x시간 단위, x시간 단위만큼 지연된다.

각 차트 쌍에는 두 가지 공통점이 있다: 시계열이 랜덤하지 않은 점과 높은 지연-1(아마 더 높은 차등화 d/D 순서가 필요할 것이다).

# 시계열분석과 Prophet을 이용한 예측

1호점의 향후 6주 예측

페이스북의 핵심 데이터 과학 팀은 최근 Predicator라고 불리는 시계열 데이터를 예측하는 새로운 절차를 발표했다. 비선형 트렌드가 연간 및 주간 계절성, 휴일에 맞는 추가 모델을 기반으로 한다.

In [ ]:
# importing data
df = pd.read_csv("../input/rossmann-store-sales/train.csv",  
                    low_memory = False)

# remove closed stores and those with no sales
df = df[(df["Open"] != 0) & (df['Sales'] != 0)]

# sales for the store number 1 (StoreType C)
sales = df[df.Store == 1].loc[:, ['Date', 'Sales']]

# reverse to the order: from 2013 to 2015
sales = sales.sort_index(ascending = False)

# to datetime64
sales['Date'] = pd.DatetimeIndex(sales['Date'])
sales.dtypes

In [ ]:
# from the prophet documentation every variables should have specific names
sales = sales.rename(columns = {'Date': 'ds',
                                'Sales': 'y'})
sales.head()

In [ ]:
# plot daily sales
ax = sales.set_index('ds').plot(figsize = (12, 4), color = c)
ax.set_ylabel('Daily Number of Sales')
ax.set_xlabel('Date')
plt.show()

# 휴일 모델링

In [ ]:
# create holidays dataframe
state_dates = df[(df.StateHoliday == 'a') | (df.StateHoliday == 'b') & (df.StateHoliday == 'c')].loc[:, 'Date'].values
school_dates = df[df.SchoolHoliday == 1].loc[:, 'Date'].values

state = pd.DataFrame({'holiday': 'state_holiday',
                      'ds': pd.to_datetime(state_dates)})
school = pd.DataFrame({'holiday': 'school_holiday',
                      'ds': pd.to_datetime(school_dates)})

holidays = pd.concat((state, school))      
holidays.head()

In [ ]:
# set the uncertainty interval to 95% (the Prophet default is 80%)
my_model = Prophet(interval_width = 0.95, 
                   holidays = holidays)
my_model.fit(sales)

# dataframe that extends into future 6 weeks 
future_dates = my_model.make_future_dataframe(periods = 6*7)

print("First week to forecast.")
future_dates.tail(7)

In [ ]:
# predictions
forecast = my_model.predict(future_dates)

# preditions for last week
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
fc = forecast[['ds', 'yhat']].rename(columns = {'Date': 'ds', 'Forecast': 'yhat'})

In [ ]:
# visualizing predicions
my_model.plot(forecast);

In [ ]:
my_model.plot_components(forecast);

첫 번째 차트는 1호점의 월 매출이 시간이 지나면서 선형적으로 감소해왔다는 것을 보여주며, 두 번째 차트는 모델에 포함된 휴일의 차이를 보여준다. 세 번째 차트는 지난주 판매량이 다음 주 월요일로 정점을 찍은 반면, 네 번째 차트는 크리스마스 연휴 동안 가장 바쁜 시즌이 발생한다는 것을 보여준다.